# Лабораторная работа №3

выполнила: Рыжкова Ульяна БВТ2001

In [1]:
import torch
import torch.nn.functional as F

In [25]:
def myconv_transpose2d(inputs, weight, bias=None, stride=1 , padding=0, output_padding=0, groups=1, dilation=1):
    # Get the dimensions of the input and kernel
    batch_size, in_channels, in_height, in_width = inputs.shape
    _, out_channels, kernel_height, kernel_width = weight.shape

    # Compute the output dimensions
    out_height = (in_height - 1) * stride - 2 * padding + dilation * (kernel_height - 1) + output_padding +1 
    out_width = (in_width - 1) * stride - 2 * padding + dilation * (kernel_width - 1) + output_padding + 1

    # Create the output tensor
    output = torch.zeros((batch_size, out_channels, out_height, out_width))

    # Apply the 2D transposed convolution
    for b in range(batch_size):
        for oc in range(out_channels):
            for oh in range(out_height):
                for ow in range(out_width):
                    for ic in range(in_channels):
                        for kh in range(kernel_height):
                            for kw in range(kernel_width):
                                i = oh + padding - kh * dilation
                                j = ow + padding - kw * dilation
                                if i >= 0 and j >= 0 and i < in_height * stride and j < in_width * stride and (i % stride == 0) and (j % stride == 0):
                                    i //= stride
                                    j //= stride
                                    output[b, oc, oh, ow] += inputs[b, ic, i, j] * weight[oc, ic, kh, kw]

            #bias
            if bias is not None:
                output[b, oc, :, :] += bias[oc]

    return output


batch_size, in_channels, height, width = 1, 1, 4, 4
out_channels = 1
kernel_size = 3

input_data = torch.randn(batch_size, in_channels, height, width)
weight_data = torch.randn(in_channels, out_channels, kernel_size, kernel_size)
bias_data = torch.randn(out_channels)

# Use function
output = myconv_transpose2d(input_data, weight_data, bias_data, stride=1, padding=1, dilation=1, groups=1)

In [3]:
import ipytest
ipytest.autoconfig()

In [40]:
%%ipytest


def test_1():
    batch_size, in_channels, height, width = 1, 1, 3, 3
    out_channels = 1
    kernel_size = 3

    input_data = torch.randn(batch_size, in_channels, height, width)
    weight_data = torch.randn(in_channels, out_channels, kernel_size, kernel_size)        


    conv = F.conv_transpose2d(input_data, weight_data, bias=None, stride=1, padding=1)
    my_conv = myconv_transpose2d(input_data, weight_data, bias=None, stride=1, padding=1)
    assert torch.allclose(conv, my_conv)

def test_2():
    batch_size, in_channels, height, width = 1, 1, 3, 3
    out_channels = 1
    kernel_size = 3

    input_data = torch.randn(batch_size, in_channels, height, width)
    weight_data = torch.randn(in_channels, out_channels, kernel_size, kernel_size)        
    bias_data = torch.randn(out_channels)

    conv = F.conv_transpose2d(input_data, weight_data, bias_data, stride=1, padding=1)
    my_conv = myconv_transpose2d(input_data, weight_data, bias_data, stride=1, padding=1)
    assert torch.allclose(conv, my_conv)


def test_3():
    batch_size, in_channels, height, width = 1, 1, 3, 3
    out_channels = 1
    kernel_size = 3

    input_data = torch.randn(batch_size, in_channels, height, width)
    weight_data = torch.randn(in_channels, out_channels, kernel_size, kernel_size)        
    bias_data = torch.randn(out_channels)

    my_conv = myconv_transpose2d(input_data, weight_data, bias_data, stride=2, padding=0)
    conv = F.conv_transpose2d(input_data, weight_data, bias_data, stride=2, padding=0)
    assert torch.allclose(conv, my_conv)

def test_4():
    batch_size, in_channels, height, width = 1, 1, 3, 3
    out_channels = 1
    kernel_size = 3
    dilation = 2

    input_data = torch.randn(batch_size, in_channels, height+dilation, width+dilation)
    weight_data = torch.randn(in_channels, out_channels, kernel_size, kernel_size)        
    bias_data = torch.randn(out_channels)

    my_conv = myconv_transpose2d(input_data, weight_data, bias_data, stride=1, padding=0, dilation=dilation)
    conv = F.conv_transpose2d(input_data, weight_data, bias_data, stride=1, padding=0, dilation=dilation)
    assert torch.allclose(conv, my_conv)

def test_5():
    batch_size, in_channels, height, width = 1, 1, 3, 3
    out_channels = 1
    kernel_size = 3

    input_data = torch.randn(batch_size, in_channels, height, width)
    weight_data = torch.randn(in_channels, out_channels, kernel_size, kernel_size)        
    bias_data = torch.randn(out_channels)

    my_conv = myconv_transpose2d(input_data, weight_data, bias_data, stride=2, padding=0, output_padding=1)
    conv = F.conv_transpose2d(input_data, weight_data, bias_data, stride=2, padding=0, output_padding=1)
    assert torch.allclose(conv, my_conv)

.....                                                                                        [100%]
5 passed in 0.07s
